# Finetuning roBERTa for Political Media Bias Detection (model 2)

## Step 1: Data pre-processing

### Import packages & libraries

In [ ]:
pip install transformers datasets torch scikit-learn pandas

In [ ]:
pip install --upgrade wandb --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 73.0 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.0
    Uninstalling transformers-4.57.0:
      Successfully uninstalled transformers-4.57.0


In [ ]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import RobertaForSequenceClassification, RobertaTokenizer
from transformers import BertTokenizer
from datasets import Dataset
from transformers import BertForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.utils.class_weight import compute_class_weight
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Load & inspect dataset

In [ ]:
# Load labeled data (for training and TTV split)
labeled_data_path = '/content/drive/My Drive/dsa4213/labelled_data_clean.csv'
df_labelled = pd.read_csv(labeled_data_path)

HTTPError: HTTP Error 401: Unauthorized

In [ ]:
# Check the first few rows of data
df_labelled.head(10)

,title,author,permalink,body,bias,bias_text
0,"Bomb Suspect Changed After Trip Abroad, Friend...",N. R. Kleinfield,http://www.nytimes.com/2016/09/20/nyregion/ahm...,"Besides his most recent trip to Quetta , Mr. R...",0,left
1,Why Susan Collins claims she’s being bribed ov...,"Emily Stewart, Terry Nguyen, Rebecca Jennings,...",https://www.vox.com/policy-and-politics/2018/9...,Is Maine Republican Sen. Susan Collins being b...,0,left
2,Poll: Prestigious Colleges Won't Make You Happ...,Anya Kamenetz,http://www.npr.org/blogs/thetwo-way/2014/05/06...,Poll : Prestigious Colleges Wo n't Make You Ha...,0,left
3,Paul Ryan Reportedly Says No Chance for Border...,Ian Mason,http://www.breitbart.com/big-government/2017/0...,"House Speaker Paul Ryan , at a private dinner ...",2,right
4,OPINION: Trump seeking change of legal fortune...,Analysis Stephen Collinson,https://www.cnn.com/2019/07/11/politics/donald...,( CNN ) President Donald Trump has reason to h...,0,left
5,PAUL: Blocking the pathway to a national ID,Sen. Rand Paul,http://www.washingtontimes.com/news/2013/may/2...,The controversial immigration-reform bill that...,2,right
6,Dick Morris Says He Is Working On An RNC Ad Ai...,NaN,http://mediamatters.org/blog/2013/03/28/dick-m...,Dick Morris is working with Republican Nationa...,0,left
7,WSJ Economist Moore: No Grounds Logic for Obam...,"Jim Meyers, John Bachman",http://www.newsmax.com/Newsfront/moore-obama-t...,Wall Street Journal economics expert Stephen M...,2,right
8,Bernie Surges,NaN,https://www.theflipside.io/archives/bernie-surges,The left believes Sanders ’ s chances have imp...,1,center
9,AOC for president? The buzz has begun,NaN,https://www.politico.com/news/2019/12/27/aoc-p...,Sanders and Ocasio-Cortez ’ s fans have also b...,0,left


In [ ]:
# Check the unique values in the 'Label' column and their counts
label_counts = df_labelled['bias_text'].value_counts()

# Display the unique labels and their counts
print(label_counts)

# Clean the 'body' column to ensure all entries are strings
df_labelled['body'] = df_labelled['body'].fillna('').astype(str)

bias_text
right     13719
left      12930
center    10791
Name: count, dtype: int64


In [ ]:
# Load unlabelled data (for prediction)
labeled_data_path = '/content/drive/My Drive/dsa4213/unlabelled_data_clean.csv'
df_unlabelled = pd.read_csv(labeled_data_path)

In [ ]:
# Check the first few rows of data
df_unlabelled.head(10)

,title,author,permalink,body
0,Nancy Pelosi Has Amassed ~$200 Million Since F...,Own_Palpitation_8477,/r/Askpolitics/comments/1hcmrgi/nancy_pelosi_h...,"As the title says, how do folks who see their ..."
1,Have the Trump supporters around you gotten qu...,SteveinTenn,/r/Askpolitics/comments/1hgqny2/have_the_trump...,Mine have suddenly lost interest in discussing...
2,With Trump banning trans people from the milit...,MisterFyre,/r/Askpolitics/comments/1hl2dpt/with_trump_ban...,NaN
3,"Elon Musk is $70,000,000,000 richer since supp...",hotdogman200,/r/Askpolitics/comments/1hcssgg/elon_musk_is_7...,"Keep in mind he is not just a donor, he is now..."
4,For all of the people who claim California is ...,Advanced_Aspect_7601,/r/Askpolitics/comments/1hn3z7a/for_all_of_the...,I've noticed California has kind of become the...
5,"Trump voters, did you believe Trump when he sa...",Snarkasm71,/r/Askpolitics/comments/1gxon1k/trump_voters_d...,Now that Donald Trump has nominated the archit...
6,"In light of Joe Biden pardoning Hunter, why di...",Hot_Cryptographer552,/r/Askpolitics/comments/1h4olop/in_light_of_jo...,NaN
7,Trump Supporters - How Are You Feeling About T...,chewbaccasaux,/r/Askpolitics/comments/1grgs4c/trump_supporte...,As an (apparently out of touch) liberal democr...
8,"Conservatives, how do you feel about trump adm...",themontajew,/r/Askpolitics/comments/1h9tu5l/conservatives_...,Now that everyone seems on the same page of ho...
9,Do people actually believe that racism and mis...,Feeling-Currency6212,/r/Askpolitics/comments/1h1kc07/do_people_actu...,For the liberals or anyone who voted for Kamal...


## Step 2: Tokenize the text and prepare the data for roBERTa

In [ ]:
# Load the roBERTa tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Preprocess function to tokenize text (using 'body' column)
def preprocess_function(examples):
    return tokenizer(examples['body'], truncation=True, padding=True, max_length=256)

# Create label mapping
print("\nActual labels in dataset:")
print(df_labelled['bias_text'].unique())

label_mapping = {'center': 0, 'left': 1, 'right': 2}

# Encode labels to integers
df_labelled['labels'] = df_labelled['bias_text'].map(label_mapping)

# Check if any labels failed to map
if df_labelled['labels'].isna().any():
    print("\n⚠️ WARNING: Some labels couldn't be mapped!")
    print("Unmapped labels:")
    print(df_labelled[df_labelled['labels'].isna()]['bias_text'].value_counts())
    print("\n🔴 Please fix the label_mapping dictionary above!")
    df_labelled = df_labelled.dropna(subset=['labels'])

# Convert to int
df_labelled['labels'] = df_labelled['labels'].astype(int)

print("\nMapped label distribution:")
print(df_labelled['labels'].value_counts().sort_index())

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]


Actual labels in dataset:
['left' 'right' 'center']

Mapped label distribution:
labels
0    10791
1    12930
2    13719
Name: count, dtype: int64


## Step 3: TTV Split

In [ ]:
# Split the labeled data into train and validation sets WITH STRATIFICATION
train_df, val_df = train_test_split(
    df_labelled,
    test_size=0.15,
    random_state=42,
    stratify=df_labelled['labels']
)

print(f"\nTrain set size: {len(train_df)}")
print("Train label distribution:")
print(train_df['labels'].value_counts().sort_index())

print(f"\nValidation set size: {len(val_df)}")
print("Validation label distribution:")
print(val_df['labels'].value_counts().sort_index())

# Convert pandas DataFrame to Hugging Face Dataset format
train_dataset = Dataset.from_pandas(train_df[['body', 'labels']])
val_dataset = Dataset.from_pandas(val_df[['body', 'labels']])

# Tokenize the datasets
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)

# Set format for PyTorch (IMPORTANT: includes 'labels')
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# For the unlabeled dataset, tokenize the 'body' column
df_unlabelled['body'] = df_unlabelled['body'].apply(str)
unlabeled_dataset = Dataset.from_pandas(df_unlabelled[['body']])
unlabeled_dataset = unlabeled_dataset.map(preprocess_function, batched=True)
unlabeled_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

# Check tokenized output (optional)
print("\nTokenized Train Dataset Example:")
print(train_dataset[0])

decoded_text = tokenizer.decode(train_dataset[0]['input_ids'])
print(f"\nDecoded text preview: {decoded_text[:200]}...")


Train set size: 31824
Train label distribution:
labels
0     9172
1    10991
2    11661
Name: count, dtype: int64

Validation set size: 5616
Validation label distribution:
labels
0    1619
1    1939
2    2058
Name: count, dtype: int64


Map:   0%|          | 0/31824 [00:00<?, ? examples/s]

Map:   0%|          | 0/5616 [00:00<?, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]


Tokenized Train Dataset Example:
{'labels': tensor(0), 'input_ids': tensor([    0,   846,  9211, 13851,   961, 11687,    14,  2455,     5,  1226,
            7,  1136,   160,     5,  2358, 15344,    74,    28,    10,  1099,
          631,   479, 50118, 28747,  1767,    74,    28,   847,  2156,  2556,
           74,  1430,  3625,    15,    10,  1647,     9,  1791,  2156,     8,
          309,     7,     5,  9588,  8587,  1387,  2156,     5,   866,    74,
         1136,   124,    88,  7306,   479, 50118,  1708,   120,    42,  4832,
         1648,   114,    70,     9,   167,   383,  1369,  2156,    89,    74,
          202,    28,    10,  1229,  3781,   479, 50118,  1779,    24,   606,
            7,  9072,     5,  2358, 15344,    93,    14,  4069,     9,   629,
         3488,     8,  1408,  2599,    14,    74,  6885,  1642,    11,   644,
         3867,  1148,     8,     5,   394,  1149,    11,    93,   752,  1229,
        25259,  8995,  6888,  7232,  4072,     7,  2422,   462, 11649,   

## Step 4: Compute class weights and use custom trainer

In [ ]:
# compute class weights since classes/ labels are imbalanced
print("COMPUTING CLASS WEIGHTS")

class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(train_df['labels']),
    y=train_df['labels']
)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float)

print(f"\nClass weights (to handle imbalance):")
for i, weight in enumerate(class_weights):
    print(f"  Class {i}: {weight:.4f}")

# custom trainer with weighted classes/ labels
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits

        # Apply class weights to loss
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights_tensor.to(logits.device), label_smoothing=0.1)
        loss = loss_fct(logits, labels)

        return (loss, outputs) if return_outputs else loss

# evaluation metrics function (simplified, overall metrics only)
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = predictions.argmax(axis=1)

    # overall metrics only
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    precision = precision_score(labels, preds, average='weighted')
    recall = recall_score(labels, preds, average='weighted')

    # optional: prediction distribution
    unique, counts = np.unique(preds, return_counts=True)
    print(f"\n📊 Prediction distribution: {dict(zip(unique, counts))}")

    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }


COMPUTING CLASS WEIGHTS

Class weights (to handle imbalance):
  Class 0: 1.1566
  Class 1: 0.9652
  Class 2: 0.9097


## Step 5: Finetune roBERTa

In [ ]:
# load roBERTa model
num_labels = len(df_labelled['labels'].unique())
print(f"\nNumber of classes: {num_labels}")

from transformers import RobertaForSequenceClassification

model = RobertaForSequenceClassification.from_pretrained(
    'roberta-base',
    num_labels=num_labels,
    problem_type="single_label_classification",
    hidden_dropout_prob=0.3,           # ADD THIS
    attention_probs_dropout_prob=0.3,  # ADD THIS
)

# finetuning parameters
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,                  # longer training for full convergence
    per_device_train_batch_size=16,       # larger batch size improves stability
    per_device_eval_batch_size=32,        # faster evaluation
    learning_rate=1e-5,                   # lower LR for stable fine-tuning
    warmup_ratio=0.1,                     # small warmup helps convergence
    weight_decay=0.1,                    # standard weight decay
    logging_strategy="epoch",             # only log per epoch
    eval_strategy="epoch",                # evaluate per epoch
    save_strategy="epoch",                # save per epoch
    save_total_limit=3,                   # keep 3 best checkpoints
    load_best_model_at_end=True,          # automatically load best model
    metric_for_best_model="f1",           # optimize for F1 score
    greater_is_better=True,
    report_to="none",
    seed=42,
    fp16=torch.cuda.is_available(),       # use mixed precision if GPU supports
    gradient_accumulation_steps=2,        # larger effective batch possible
    dataloader_num_workers=2,             # speed up data loading
    remove_unused_columns=False,          # avoid data loss
)

# initialise weighted trainer
trainer = WeightedTrainer(  # or FocalLossTrainer
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# start training
trainer.train()

# save model to gdrive
model.save_pretrained('/content/drive/My Drive/dsa3101/bias_detection_model_roberta')
tokenizer.save_pretrained('/content/drive/My Drive/dsa3101/bias_detection_model_roberta')

print("\n✓ Model and tokenizer saved to Google Drive!")


Number of classes: 3


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,1.035200,0.847844,0.642272,0.640751,0.674614,0.642272
2,0.771700,0.752037,0.740563,0.738905,0.761976,0.740563
3,0.686800,0.771974,0.732372,0.725588,0.772600,0.732372
4,0.644700,0.748901,0.759615,0.757259,0.784389,0.759615
5,0.619400,0.794692,0.752493,0.748598,0.790119,0.752493
6,0.597900,0.802318,0.746973,0.740044,0.785770,0.746973
7,0.580700,0.780120,0.758191,0.753033,0.788186,0.758191



📊 Prediction distribution: {np.int64(0): np.int64(2431), np.int64(1): np.int64(1814), np.int64(2): np.int64(1371)}

📊 Prediction distribution: {np.int64(0): np.int64(1420), np.int64(1): np.int64(2641), np.int64(2): np.int64(1555)}

📊 Prediction distribution: {np.int64(0): np.int64(1639), np.int64(1): np.int64(2781), np.int64(2): np.int64(1196)}

📊 Prediction distribution: {np.int64(0): np.int64(1467), np.int64(1): np.int64(2671), np.int64(2): np.int64(1478)}

📊 Prediction distribution: {np.int64(0): np.int64(1495), np.int64(1): np.int64(2808), np.int64(2): np.int64(1313)}

📊 Prediction distribution: {np.int64(0): np.int64(1687), np.int64(1): np.int64(2728), np.int64(2): np.int64(1201)}

📊 Prediction distribution: {np.int64(0): np.int64(1634), np.int64(1): np.int64(2671), np.int64(2): np.int64(1311)}

✓ Model and tokenizer saved to Google Drive!


## Step 6: Evaluate model performance

In [ ]:
# evaluate model performance
print("FINAL EVALUATION")
results = trainer.evaluate()
print("\nEvaluation Results:", results)

# prediction check/ verification
print("PREDICTION SANITY CHECK")

# Get predictions on validation set
predictions = trainer.predict(val_dataset)
preds = predictions.predictions.argmax(axis=1)

print("\nValidation set PREDICTION distribution:")
unique, counts = np.unique(preds, return_counts=True)
for label, count in zip(unique, counts):
    percentage = count / len(preds) * 100
    print(f"  Class {label}: {count} ({percentage:.1f}%)")

print("\nValidation set TRUE label distribution:")
true_labels = val_df['labels'].values
unique, counts = np.unique(true_labels, return_counts=True)
for label, count in zip(unique, counts):
    percentage = count / len(true_labels) * 100
    print(f"  Class {label}: {count} ({percentage:.1f}%)")

if len(np.unique(preds)) < num_labels:
    print("\n🔴 WARNING: Model is not predicting all classes!")
    print("   The model has collapsed to predicting only majority classes.")
    print("   Consider:")
    print("   1. Increasing class weights further")
    print("   2. Oversampling minority classes")
    print("   3. Using focal loss")
else:
    print("\n✅ SUCCESS: Model is predicting all classes!")

FINAL EVALUATION



📊 Prediction distribution: {np.int64(0): np.int64(1467), np.int64(1): np.int64(2671), np.int64(2): np.int64(1478)}

Evaluation Results: {'eval_loss': 0.748900830745697, 'eval_accuracy': 0.7596153846153846, 'eval_f1': 0.7572593360480849, 'eval_precision': 0.784389088287231, 'eval_recall': 0.7596153846153846, 'eval_runtime': 19.086, 'eval_samples_per_second': 294.248, 'eval_steps_per_second': 9.221, 'epoch': 7.0}
PREDICTION SANITY CHECK

📊 Prediction distribution: {np.int64(0): np.int64(1467), np.int64(1): np.int64(2671), np.int64(2): np.int64(1478)}

Validation set PREDICTION distribution:
  Class 0: 1467 (26.1%)
  Class 1: 2671 (47.6%)
  Class 2: 1478 (26.3%)

Validation set TRUE label distribution:
  Class 0: 1619 (28.8%)
  Class 1: 1939 (34.5%)
  Class 2: 2058 (36.6%)

✅ SUCCESS: Model is predicting all classes!


## Step 7: Make predictions on unlabelled data

In [ ]:
# Make predictions
predictions_unlabeled = trainer.predict(unlabeled_dataset)
predicted_labels = predictions_unlabeled.predictions.argmax(axis=1)
predicted_probs = torch.softmax(torch.tensor(predictions_unlabeled.predictions), dim=1).numpy()

# Convert back to label names
reverse_label_mapping = {v: k for k, v in label_mapping.items()}
df_unlabelled['predicted_label'] = [reverse_label_mapping[label] for label in predicted_labels]

# Add confidence scores
df_unlabelled['confidence'] = predicted_probs.max(axis=1)

# Add probabilities for each class
for i, label_name in enumerate(['neutral', 'left', 'right']):
    df_unlabelled[f'prob_{label_name}'] = predicted_probs[:, i]

# Show prediction distribution
print("\nPrediction distribution on unlabeled data:")
print(df_unlabelled['predicted_label'].value_counts())

print(f"\nAverage confidence: {df_unlabelled['confidence'].mean():.4f}")

# Save predictions
df_unlabelled.to_csv('/content/drive/My Drive/dsa3101/predictions.csv', index=False)
print("\n✓ Predictions saved to 'predictions.csv'!")

# Show some examples
print("\nSample predictions:")
print(df_unlabelled[['body', 'predicted_label', 'confidence']].head(10))


Prediction distribution on unlabeled data:
predicted_label
left      77
center    40
right     33
Name: count, dtype: int64

Average confidence: 0.6925

✓ Predictions saved to 'predictions.csv'!

Sample predictions:
                                                body predicted_label  \
0  As the title says, how do folks who see their ...            left   
1  Mine have suddenly lost interest in discussing...            left   
2                                                nan            left   
3  Keep in mind he is not just a donor, he is now...           right   
4  I've noticed California has kind of become the...          center   
5  Now that Donald Trump has nominated the archit...            left   
6                                                nan            left   
7  As an (apparently out of touch) liberal democr...            left   
8  Now that everyone seems on the same page of ho...           right   
9  For the liberals or anyone who voted for Kamal...           